### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. 

Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.


По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

На примере сайта Superjob:

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
url = 'https://russia.superjob.ru/vacancy/search/'
jobname = 'Тестировщик'
params = {
        'keywords': jobname,
        'profession_only': 1,
        'page': ''
    }

In [3]:
vacancies = {'city': [],
             'name': [], 
             'company': [], 
             'salary_min': [], 
             'salary_max': [], 
             'link': [], 
             'site': []
            }

response = requests.get(url, params=params).text
html = bs(response, 'html.parser')
pagination = html.find('a', {'class': 'f-test-button-1'})

if not pagination:
    last_page = 1
else:
    pagination = pagination.findParent()
    last_page = int(pagination.find_all('a')[-2].text)

for page in range(1, last_page + 1):
    params['page'] = page
    response = requests.get(url, params=params).text
    html = bs(response, 'html.parser')
    if html.find_all(attrs={'class': 'f-test-vacancy-item'}):
        results = html.find_all(attrs={'class': 'f-test-vacancy-item'})
        for result in results:
            # city
            city = result.find(attrs={'class': 'f-test-text-company-item-location'}).findChildren()[2].text.split(',')[0]
            vacancies['city'].append(city)
            
            # job title
            vacancies['name'].append(result.find('a', href=True).text)
            
            # job link
            vacancies['link'].append('https://www.superjob.ru' + result.find('a', href=True)['href'])
            
            # company name
            company = result.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}).text if \
                result.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}) else None
            vacancies['company'].append(company)
            
            # job site
            vacancies['site'].append('Superjob')
            
            # salary
            salary = result.find(attrs={'class': 'f-test-text-company-item-salary'}).next.text
            salary = re.split(r'\s|-', salary)
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1]+salary[2])
            elif salary[0] == 'от':
                salary_min = int(salary[1]+salary[2])
                salary_max = None
            elif salary[0] == 'По':
                salary_min = None
                salary_max = None
            else:
                salary_min = int(salary[0]+salary[1])
                salary_max = int(salary[3]+salary[4]) if len(salary) > 3 else salary_min
            vacancies['salary_min'].append(salary_min)
            vacancies['salary_max'].append(salary_max)

In [4]:
df = pd.DataFrame.from_dict(vacancies)
df

,city,name,company,salary_min,salary_max,link,site
0,Москва,Тестировщик,"Предприятие ""АЛИДИ""",80000.0,90000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
1,Зеленогорск (Санкт-Петербург),Тестировщик,Сбербанк России,NaN,NaN,https://www.superjob.ru/vakansii/testirovschik...,Superjob
2,Москва,Системный аналитик / Тестировщик,Российские космические системы,NaN,NaN,https://www.superjob.ru/vakansii/sistemnyj-ana...,Superjob
3,Москва,Тестировщик ПО и радиоустройств,Смартико,NaN,105000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
4,Москва,Тестировщик ПО,Делаем бизнес,NaN,120000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
...,...,...,...,...,...,...,...
102,Санкт-Петербург,QA automation engineer,Сбербанк России,NaN,NaN,https://www.superjob.ru/vakansii/qa-automation...,Superjob
103,Брянск,Инженер по функциональному тестированию,Сбербанк России,NaN,NaN,https://www.superjob.ru/vakansii/inzhener-po-f...,Superjob
104,Москва,Главный инженер по тестированию,Сбербанк России,NaN,NaN,https://www.superjob.ru/vakansii/glavnyj-inzhe...,Superjob
105,Москва,Junior QA Automation Engineer,Лидер тестирования,50000.0,NaN,https://www.superjob.ru/vakansii/junior-qa-aut...,Superjob
